## 추천 시스템

### 추천 시스템의 중요성
1. 추천 시스템은 이커머스의 필수 요소 -> 사용자에게 맞춤 콘텐츠를 제공
2. 추천엔진의 필요성: 사용자가 무엇은 원하는지 빠르게 찾아내어 사용자의 온라인 경험을 향상시킨다.
3. 추천시스템으로 인해 나도 모르고있었는데 나의 취향에 대한 한계폭을 넓혀 줄때, 아하! 모멘트가 생김<br>
아하 모멘트가 생기면 추천 시스템에 대한 신뢰도가 올라가고, 추천 시스템에 양질의 데이터를 제공하게 되며 선순환을 일으킨다.

### 추천 시스템 방식
1. 콘텐츠 기반 필터링: 사용자에게 맞는 콘텐츠를 제공<br>
예를 들어, 영화 콘텐츠인 경우, 영화의 감독, 배우, 영화 내용을 개인의 취향에 반영해서 판별해 주는 것.
2. 협업 필터링: 사용자에게 비슷한 사람이 선택한 상품을 추천해줌.<br>
예를 들어, 나도 모르고있었는데 나와 비슷한 사람들이 선택한 상품들을 추천해줌.

**-> 이 둘 중 하나를 사용하거나, 둘을 결합한 하이브리드 방식을 사용한다.**

## 9.1 컨텐츠 기반 필터링 실습
**콘텐츠 기반 필터링 구현 프로세스**
1. 콘텐츠에 대한 여러 텍스트 정보들을 피처 벡터화
2. 코사인 유사도로 콘텐츠별 유사도 계산
3. 콘텐츠 별로 유사도가 높은것에 대해 가중 평점을 계산
4. 유사도가 높은 콘텐츠 중에 평점이 좋은 콘텐츠 순으로 추천

### 콘텐츠 기반 필터링 예시 - 영화
ARRIVAL과 프로메테우스에 좋은 평점을 준 사용자에게 감독, 출연, 장르, 내용, 비평가 평등을 고려해서<br>
원작이 리들리 스콧이고, 감독이 드니 빌뇌브인 블레이드 러너를 추천해주는것
<br><img src=https://blog.kakaocdn.net/dn/cjYwVQ/btrGF7dnQLg/ygyqWR0i0fiQqeJuJiuXSk/img.png width=1000>

### 콘텐츠 기반 필터링 과정
영화 구성 콘텐츠를 모두 텍스트화 -> 피처 벡터화 -> 코사인 유사도 -> 유사도에 따른 추천<br>
<br><img src=https://blog.kakaocdn.net/dn/8NWzl/btrGF7LcXLV/klljDlpDHr18QqkVKt87J0/img.png width=1000>

### TMDB 5000 Movie Dataset 데이터 로드
https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata<br>
TMDB 5000: 미국에서 유명한 영화 평점사이트 IMDB에서 영화 5000개에 대한 메타데이터셋<br>
<br><img src=https://blog.kakaocdn.net/dn/zDZgQ/btrGI0dgD6W/Yi0ec4gn4xHkUHMzEXR8z1/img.png width=700>

In [19]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies =pd.read_csv('./tmdb-5000-movie-dataset/tmdb_5000_movies.csv')
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


### 필요컬럼만 추출

In [21]:
movies_df = movies[['id','title', 'genres', 'vote_average', 'vote_count',
                 'popularity', 'keywords', 'overview']]

In [22]:
pd.set_option('max_colwidth', 100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [4]:
# genres가 모두 text(object)이다
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 8 columns):
id              4803 non-null int64
title           4803 non-null object
genres          4803 non-null object
vote_average    4803 non-null float64
vote_count      4803 non-null int64
popularity      4803 non-null float64
keywords        4803 non-null object
overview        4800 non-null object
dtypes: float64(2), int64(2), object(4)
memory usage: 300.3+ KB


### 텍스트 가공 (genres, keywords 피처)
**텍스트 문자 1차 가공. 파이썬 딕셔너리 변환 후 리스트 형태로 변환**

In [23]:
# literal_eval: 문자를 가공해서 알맞은 python의 객체를 만들어주는 모듈
from ast import literal_eval # 여기선 문자열을 dict의 list로 만들어줌

movies_df['genres'] = movies_df['genres'].apply(literal_eval)
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [24]:
movies_df['genres'].head(1)

0    [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {...
Name: genres, dtype: object

In [26]:
# apply lambda를 사용 -> name만 가져오기 
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x]) # 리스트(x)를 반복하며, dict(y)에서 'name'의 값만 가져오기
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
movies_df[['genres', 'keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


### genres: Count벡터화 -> 코사인 유사도 비교
**장르 콘텐츠 필터링을 이용한 영화 추천. 장르 문자열을 Count 벡터화 후에 코사인 유사도로 각 영화를 비교**<br>
**genres Count기반 피처 벡터화 -> genres_literal**

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer를 적용하기 위해 genres를 ' '.join()해서 공백문자로 결합
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(f"genres를 하나의 문자열로 합치고, Count 벡터화한 결과 shape: {genre_mat.shape}")

genres를 하나의 문자열로 합치고, Count 벡터화한 결과 shape: (4803, 276)


In [29]:
# genres_literal 컬럼이 genres를 하나의 문자열로 합친 컬럼
movies_df.head(2)

,id,title,genres,vote_average,vote_count,popularity,keywords,overview,genres_literal
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]",7.2,11800,150.437577,"[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa...","In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",Action Adventure Fantasy Science Fiction
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]",6.9,4500,139.082615,"[ocean, drug abuse, exotic island, east india trading company, love of one's life, traitor, ship...","Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of t...",Adventure Fantasy Action


### 코사인 유사도
**장르에 따른 영화별 코사인 유사도 추출**

In [44]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:2])

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]]


In [49]:
genre_sim.argsort()

array([[2401, 3037, 3038, ...,  813, 3494,    0],
       [2401, 3067, 3069, ...,  129,    1,  262],
       [2401, 2999, 3000, ..., 1542, 1740,    2],
       ...,
       [   0, 2230, 2229, ..., 1895, 3809, 4800],
       [   0, 3205, 3204, ..., 1596, 1594, 4802],
       [   0, 3141, 3140, ..., 4521, 4710, 4802]])

In [64]:
genre_sim.argsort()

array([[2401, 3037, 3038, ...,  813, 3494,    0],
       [2401, 3067, 3069, ...,  129,    1,  262],
       [2401, 2999, 3000, ..., 1542, 1740,    2],
       ...,
       [   0, 2230, 2229, ..., 1895, 3809, 4800],
       [   0, 3205, 3204, ..., 1596, 1594, 4802],
       [   0, 3141, 3140, ..., 4521, 4710, 4802]])

In [68]:
genre_sim.argsort()[:, ::-1]

array([[   0, 3494,  813, ..., 3038, 3037, 2401],
       [ 262,    1,  129, ..., 3069, 3067, 2401],
       [   2, 1740, 1542, ..., 3000, 2999, 2401],
       ...,
       [4800, 3809, 1895, ..., 2229, 2230,    0],
       [4802, 1594, 1596, ..., 3204, 3205,    0],
       [4802, 4710, 4521, ..., 3140, 3141,    0]])

In [54]:
## 각 row별로 내림차순 정렬해서 인덱스 추출
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(f"첫번째 row 출력\n{genre_sim_sorted_ind[:1]}")

첫번째 row 출력
[[   0 3494  813 ... 3038 3037 2401]]


#### 특정 영화(title_name) - 유사도 반환함수
**특정 영화와 장르별 유사도가 높은 영화를 반환하는 함수 생성**

In [63]:
movies_df[movies_df.title=='The Godfather'].index.values

array([3337])

In [72]:
genre_sim_sorted_ind[3]

array([2195, 1850, 3316, ...,  887, 2544, 4802])

In [55]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    
    # 인자로 입력된 movies_df DataFrame에서 'title' 컬럼이 입력된 title_name 값인 DataFrame추출
    title_movie = df[df['title'] == title_name]
    
    # title_name을 가진 DataFrame의 index 객체를 ndarray로 반환하고 
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n 개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)] # sorted_ind에서 title_index번째 행이 해당 영화에 대한 cosine 유사도 행(관련도 내림차순)
    
    # 추출된 top_n index들 출력. top_n index는 2차원 데이터 임. 
    print(similar_indexes) # df에서 해당 행들이 순서대로 title_name과 가장 가까운 영화들
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes] # 데이터프레임으로 반환

#### 'The Godfather'에 대한, 유사도 반환

In [74]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',15)
similar_movies[['title', 'vote_average']]
# 14번째가 자기 자신

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866 3112 4041  588 3337
  3378]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


### 평가 횟수에 가중치 부여

#### **평점이 높은 영화 정보 확인**
: 점수가 높은 영화들이 1~2회 평가된 영화들이 많다 -> 신빙성이 떨어진다.

In [75]:
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


#### 가중치 부여 계산

**평가 횟수에 대한 가중치가 부여된 평점(Weighted Rating) 계산  
         가중 평점(Weighted Rating) = (v/(v+m)) * R + (m/(v+m)) * C**
         
■ v: 개별 영화에 평점을 투표한 횟수<br>
■ m: 평점을 부여하기 위한 최소 투표 횟수<br>
■ R: 개별 영화에 대한 평균 평점.<br>
■ C: 전체 영화에 대한 평균 평점

In [83]:
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6) # 상위 40%
print(f"평균 평점(C): {round(C,3)}\n투표 횟수 하위 60%(상위 40%)(m): {round(m,3)} 회")

평균 평점(C): 6.092
투표 횟수 하위 60%(상위 40%)(m): 370.2 회


In [85]:
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile) # 상위 40%(하위 60%)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1) 

In [86]:
movies_df[['title','vote_average','weighted_vote','vote_count']].sort_values('weighted_vote',
                                                                          ascending=False)[:10]
# 장르에 상관없이 weighted_vote(가중치 vote) 내림차순 -> 명작 영화들이 많이 나오네

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


#### 가중치 고려 장르 비슷한 영화 추천 함수
가중치 고려해서, 장르 코사인 유사도 비슷한 상위함수 10개 반환하는 함수 생성

In [88]:
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 
    similar_indexes = sorted_ind[title_index, :(top_n*2)] # 후보군을 top_n의 2배로 가져감
    similar_indexes = similar_indexes.reshape(-1)
    # 기준 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

In [89]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title', 'vote_average', 'weighted_vote']]
# Gangster 영화들 중에 평점이 높은 것들이 많이 나왔다.

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
